In [1]:
import numpy as np
import random
import math
import os
import pandas as pd
import logging

from sklearn.metrics import f1_score, matthews_corrcoef
from tensorboardX import SummaryWriter

from tqdm import tqdm, trange
from datasets import my_collate, my_collate_elmo, load_datasets_and_vocabs
from tree import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [2]:
class Parameters:
    def __init__(self):
        pass
    
class Highway(nn.Module):
    def __init__(self, layer_num, dim):
        super().__init__()
        self.layer_num = layer_num
        self.linear = nn.ModuleList([nn.Linear(dim, dim)
                                     for _ in range(layer_num)])
        self.gate = nn.ModuleList([nn.Linear(dim, dim)
                                   for _ in range(layer_num)])

    def forward(self, x):
        for i in range(self.layer_num):
            gate = F.sigmoid(self.gate[i](x))
            nonlinear = F.relu(self.linear[i](x))
            x = gate * nonlinear + (1 - gate) * x
        return x
    
def mask_logits(target, mask):
    return target * mask + (1 - mask) * (-1e30)

In [3]:
class RelationAttention(nn.Module):
    def __init__(self, in_dim = 300, hidden_dim = 64):
        # in_dim: the dimension fo query vector
        super().__init__()

        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, feature, dep_tags_v, dmask):
        '''
        C feature/context [N, L, D]
        Q dep_tags_v          [N, L, D]
        mask dmask          [N, L]
        '''
        Q = self.fc1(dep_tags_v)
        Q = self.relu(Q)
        Q = self.fc2(Q)  # (N, L, 1)
        Q = Q.squeeze(2)
        Q = F.softmax(mask_logits(Q, dmask), dim=1)

        Q = Q.unsqueeze(2)
        out = torch.bmm(feature.transpose(1, 2), Q)
        out = out.squeeze(2)
        # out = F.sigmoid(out)
        return out  # ([N, L])
    
    
class DotprodAttention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, feature, aspect_v, dmask):
        '''
        C feature/context [N, L, D]
        Q dep_tags_v          [N, D]
        mask dmask          [N, L]
        '''

        Q = aspect_v
        Q = Q.unsqueeze(2)  # (N, D, 1)
        dot_prod = torch.bmm(feature, Q)  # (N, L, 1)
        dmask = dmask.unsqueeze(2)  # (N, D, 1)
        attention_weight = mask_logits(dot_prod, dmask)  # (N, L ,1)
        attention = F.softmax(attention_weight, dim=1)  # (N, L, 1)

        out = torch.bmm(feature.transpose(1, 2), attention)  # (N, D, 1)
        out = out.squeeze(2)
        # out = F.sigmoid(out)
        # (N, D), ([N, L]), (N, L, 1)
        return out

In [4]:
class GAT_Aspect(nn.Module):
    """
    Full model in reshaped tree
    """
    def __init__(self, args, dep_tag_num, pos_tag_num):
        super(GAT_Aspect, self).__init__()
        self.args = args

        num_embeddings, embed_dim = args.glove_embedding.shape
        self.embed = nn.Embedding(num_embeddings, embed_dim)
        self.embed.weight = nn.Parameter(
            args.glove_embedding, requires_grad=False)

        self.dropout = nn.Dropout(args.dropout)
        self.tanh = nn.Tanh()

        if args.highway:
            print("Inside highway")
            self.highway_dep = Highway(args.num_layers, args.embedding_dim)
            self.highway = Highway(args.num_layers, args.embedding_dim)

        self.bilstm = nn.LSTM(input_size=args.embedding_dim, hidden_size=args.hidden_size,
                              bidirectional=True, batch_first=True, num_layers=args.num_layers)
        gcn_input_dim = args.hidden_size * 2

        self.gat_dep = [RelationAttention(in_dim = args.embedding_dim).to(args.device) for i in range(args.num_heads)]
        self.gat = [DotprodAttention().to(args.device) for i in range(args.num_heads)]

        self.dep_embed = nn.Embedding(dep_tag_num, args.embedding_dim)

        last_hidden_size = args.hidden_size * 4

        layers = [
            nn.Linear(last_hidden_size, args.final_hidden_size), nn.ReLU()]
        for _ in range(args.num_mlps-1):
            layers += [nn.Linear(args.final_hidden_size,
                                 args.final_hidden_size), nn.ReLU()]
        self.fcs = nn.Sequential(*layers)
        self.fc_final = nn.Linear(args.final_hidden_size, args.num_classes)

    def forward(self, sentence, aspect, pos_class, dep_tags, text_len, aspect_len, dep_rels, dep_heads, aspect_position, dep_dirs):
        '''
        Forward takes:
            sentence: sentence_id of size (batch_size, text_length)
            aspect: aspect_id of size (batch_size, aspect_length)
            pos_class: pos_tag_id of size (batch_size, text_length)
            dep_tags: dep_tag_id of size (batch_size, text_length)
            text_len: (batch_size,) length of each sentence
            aspect_len: (batch_size, ) aspect length of each sentence
            dep_rels: (batch_size, text_length) relation
            dep_heads: (batch_size, text_length) which node adjacent to that node
            aspect_position: (batch_size, text_length) mask, with the position of aspect as 1 and others as 0
            dep_dirs: (batch_size, text_length) the directions each node to the aspect
        '''
        fmask = (torch.zeros_like(sentence) != sentence).float()  # (N，L)
        dmask = (torch.zeros_like(dep_tags) != dep_tags).float()  # (N ,L)

        feature = self.embed(sentence)  # (N, L, D)
        aspect_feature = self.embed(aspect) # (N, L', D)
        feature = self.dropout(feature)
        aspect_feature = self.dropout(aspect_feature)

        if self.args.highway:
            feature = self.highway(feature)
            aspect_feature = self.highway(aspect_feature)

        feature, _ = self.bilstm(feature) # (N,L,D)
        aspect_feature, _ = self.bilstm(aspect_feature) #(N,L,D)

        aspect_feature = aspect_feature.mean(dim = 1) # (N, D)

        ############################################################################################
        # do gat thing
        dep_feature = self.dep_embed(dep_tags)
        if self.args.highway:
            dep_feature = self.highway_dep(dep_feature)

        dep_out = [g(feature, dep_feature, fmask).unsqueeze(1) for g in self.gat_dep]  # (N, 1, D) * num_heads
        dep_out = torch.cat(dep_out, dim = 1) # (N, H, D)
        dep_out = dep_out.mean(dim = 1) # (N, D)

        gat_out = [g(feature, aspect_feature, fmask).unsqueeze(1) for g in self.gat]
        gat_out = torch.cat(gat_out, dim=1)
        gat_out = gat_out.mean(dim=1)

        feature_out = torch.cat([dep_out,  gat_out], dim = 1) # (N, D')
        # feature_out = gat_out
        #############################################################################################
        x = self.dropout(feature_out)
        x = self.fcs(x)
        logit = self.fc_final(x)
        return logit

In [5]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)


def get_input_from_batch(args, batch):
    # sentence_ids, aspect_ids, dep_tag_ids, pos_class, text_len, aspect_len, sentiment, dep_rel_ids, dep_heads, aspect_positions
    inputs = {  'sentence': batch[0],
                'aspect': batch[1], # aspect token
                'dep_tags': batch[2], # reshaped
                'pos_class': batch[3],
                'text_len': batch[4],
                'aspect_len': batch[5],
                'dep_rels': batch[7], # adj no-reshape
                'dep_heads': batch[8],
                'aspect_position': batch[9],
                'dep_dirs': batch[10]
                }
    labels = batch[6]
    return inputs, labels


def get_collate_fn(args):
    embedding_type = args.embedding_type
    if embedding_type == 'glove':
        return my_collate


In [6]:
def train(args, train_dataset, model, test_dataset):
    '''Train the model'''
    tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size
    train_sampler = RandomSampler(train_dataset)
    collate_fn = get_collate_fn(args)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler,
                                  batch_size=args.train_batch_size,
                                  collate_fn=collate_fn)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (
            len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(
            train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    
    
    parameters = filter(lambda param: param.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=args.learning_rate)

    # Train
    print("***** Running training *****")
    print("Num examples: {}; Epochs: {}; Gradient Accumulation steps: {}; Total optimization steps: {}".format(len(train_dataset), args.num_train_epochs, args.gradient_accumulation_steps, t_total))

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    all_eval_results = []
    model.zero_grad()
    train_iterator = trange(int(args.num_train_epochs), desc="Epoch")
    set_seed(args)
    for _ in train_iterator:
        # epoch_iterator = tqdm(train_dataloader, desc='Iteration')
        for step, batch in enumerate(train_dataloader):
            model.train()
            batch = tuple(t.to(args.device) for t in batch)

            inputs, labels = get_input_from_batch(args, batch)
            #print(inputs['sentence'])
            
            #print(inputs.shape, labels.shape, batch.shape)
            logit = model(**inputs)
            loss = F.cross_entropy(logit, labels)

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()
            torch.nn.utils.clip_grad_norm_(
                model.parameters(), args.max_grad_norm)

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                # scheduler.step()  # Update learning rate schedule
                optimizer.step()
                model.zero_grad()
                global_step += 1

                # Log metrics
                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    results, eval_loss = evaluate(args, test_dataset, model)
                    all_eval_results.append(results)
                    for key, value in results.items():
                        tb_writer.add_scalar(
                            'eval_{}'.format(key), value, global_step)
                    tb_writer.add_scalar('eval_loss', eval_loss, global_step)
                    # tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar(
                        'train_loss', (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                # Save model checkpoint

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            epoch_iterator.close()
            break

    tb_writer.close()
    return global_step, tr_loss/global_step, all_eval_results


def evaluate(args, eval_dataset, model):
    results = {}

    args.eval_batch_size = args.per_gpu_eval_batch_size
    eval_sampler = SequentialSampler(eval_dataset)
    collate_fn = get_collate_fn(args)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler,
                                 batch_size=args.eval_batch_size,
                                 collate_fn=collate_fn)

    # Eval
    print("***** Running evaluation *****")
    print("Num examples: {}; Batch size: {}".format(len(eval_dataset), args.eval_batch_size))
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in eval_dataloader:
    # for batch in tqdm(eval_dataloader, desc='Evaluating'):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)
        with torch.no_grad():
            inputs, labels = get_input_from_batch(args, batch)

            logits = model(**inputs)
            tmp_eval_loss = F.cross_entropy(logits, labels)

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = labels.detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(
                out_label_ids, labels.detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    preds = np.argmax(preds, axis=1)
    # print(preds)
    result = compute_metrics(preds, out_label_ids)
    results.update(result)

    output_eval_file = os.path.join(args.output_dir, 'eval_results.txt')
    with open(output_eval_file, 'a+') as writer:
        print('***** Eval results *****')
        print("eval loss: {}".format(eval_loss))
        for key in sorted(result.keys()):
            print("{} = {}".format(key, str(result[key])))
    return results, eval_loss


def evaluate_badcase(args, eval_dataset, model, word_vocab):

    eval_sampler = SequentialSampler(eval_dataset)
    collate_fn = get_collate_fn(args)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler,
                                 batch_size=1,
                                 collate_fn=collate_fn)

    # Eval
    badcases = []
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in eval_dataloader:
    # for batch in tqdm(eval_dataloader, desc='Evaluating'):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)
        with torch.no_grad():
            inputs, labels = get_input_from_batch(args, batch)

            logits = model(**inputs)

        pred = int(np.argmax(logits.detach().cpu().numpy(), axis=1)[0])
        label = int(labels.detach().cpu().numpy()[0])
        if pred != label:
            sent_ids = inputs['sentence'][0].detach().cpu().numpy()
            aspect_ids = inputs['aspect'][0].detach().cpu().numpy()
            case = {}
            case['sentence'] = ' '.join([word_vocab['itos'][i] for i in sent_ids])
            case['aspect'] = ' '.join([word_vocab['itos'][i] for i in aspect_ids])
            case['pred'] = pred
            case['label'] = label
            badcases.append(case)

    return badcases

In [7]:
def simple_accuracy(preds, labels):
    return (preds == labels).mean()


def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds, average='macro')
    return {
        "acc": acc,
        "f1": f1
    }


def compute_metrics(preds, labels):
    return acc_and_f1(preds, labels)


In [8]:
parameters = Parameters()
# Required parameters
parameters.dataset_name = "rest"
parameters.output_dir = '/data/output-gcn'
parameters.num_classes = 3
parameters.seed = 2019


# Model parameters
parameters.glove_dir = "glove"
parameters.num_layers = 2
parameters.add_non_connect = True
parameters.multi_hop = True
parameters.max_hop = 4
parameters.num_heads = 7
parameters.dropout = 0.8
parameters.num_gcn_layers = 1
parameters.gcn_mem_dim = 300
parameters.gcn_dropout = 0.2
parameters.highway = 'store_true'

# GAT
parameters.gat_attention_type = 'dotprod'
parameters.embedding_type = 'glove'
parameters.embedding_dim = 300
parameters.dep_relation_embed_dim = 300    
parameters.hidden_size = 300 
parameters.final_hidden_size = 300 
parameters.num_mlps = 2

# Training parameters
parameters.per_gpu_train_batch_size = 16
parameters.per_gpu_eval_batch_size = 32
parameters.gradient_accumulation_steps = 2
parameters.learning_rate = 1e-3
parameters.weight_decay = 0.0
parameters.adam_epsilon = 1e-8
parameters.max_grad_norm = 1.0
parameters.num_train_epochs = 30.0
parameters.max_steps = -1
parameters.logging_steps = 50

## Training the Model

In [ ]:
# Setup CUDA, GPU training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
parameters.device = device
print('Device is {}'.format(parameters.device))

# Load datasets and vocabs
train_dataset, test_dataset, word_vocab, dep_tag_vocab, pos_tag_vocab= load_datasets_and_vocabs(parameters)

print(dep_tag_vocab)

# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

model = GAT_Aspect(parameters, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree

model.to(parameters.device)
# Train
_, _,  all_eval_results = train(parameters, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    for key in sorted(best_eval_result.keys()):
        print("  %s = %s", key, str(best_eval_result[key]))

Device is cuda
# Read %s Train set: %d rest 1978
# Read %s Test set: %d rest 600
*** Start processing data(unrolling and reshaping) ***
Total sentiment counter: %s defaultdict(<class 'int'>, {'negative': 805, 'positive': 2164, 'neutral': 633})
Multi-Aspect-Multi-Sentiment counter: %s defaultdict(<class 'int'>, {'positive': 351, 'neutral': 288, 'negative': 301})
*** Start processing data(unrolling and reshaping) ***
Total sentiment counter: %s defaultdict(<class 'int'>, {'positive': 728, 'neutral': 196, 'negative': 196})
Multi-Aspect-Multi-Sentiment counter: %s defaultdict(<class 'int'>, {'positive': 85, 'neutral': 83, 'negative': 60})
****** After unrolling ******
Train set size: %s 3602
Test set size: %s, 1120
{'itos': ['<pad>', '<unk>', 'non-connect', 'ncon_3', 'ncon_2', 'ncon_4', 'det', 'nsubj', 'amod', 'conj', 'pobj', 'dobj', 'cc', 'prep', 'dep', 'poss', 'nn', 'advmod', 'rcmod', 'appos', 'cop', 'nsubjpass', 'num', 'partmod', 'aux', 'ccomp', 'advcl', 'predet', 'auxpass', 'infmod', '

Epoch:   0%|                                                                                    | 0/30 [00:00<?, ?it/s]C:\Users\harsh\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
